In [ ]:
# TODO Clean and compress.

In [1]:
import pandas as pd
import pyodbc
from datetime import date
import download_datasets as dd

In [2]:
dd.csv_fetcher()
today = date.today()
name = str(today) + "-owid-covid-data.csv"
df = pd.read_csv(name)

Files will be saved in c:\Users\tomas\OneDrive\Documents\CODE\Data Analysis\SQL Server\Project Covid.
2022-01-24-owid-covid-data.csv save completed.


In [3]:
df.continent = df.continent.fillna('NULL')
rslt_df = df[df.continent != 'NULL']
loc_df = rslt_df.drop_duplicates(subset = ['location'])

In [4]:
def conection():
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server}; \
        SERVER=DESKTOP-2KNL3QE; \
        DATABASE=Portfolio_Project_Covid; \
        Trusted_Connection=yes;')
    return conn
conn = conection()
cursor = conn.cursor()

Locations

In [5]:
cursor.execute("DROP TABLE IF EXISTS locations; \
                CREATE TABLE locations \
                (iso_code varchar(255), \
                continent varchar(255), \
                location varchar(255))")

In [6]:
for index, row in loc_df.iterrows():
    cursor.execute("INSERT INTO dbo.locations([iso_code], [continent], [location]) VALUES(?, ?, ?)",
    row['iso_code'],
    row['continent'],
    row['location'])
conn.commit()

Country Data

In [7]:
cursor.execute("DROP TABLE IF EXISTS country_data; \
                CREATE TABLE country_data \
                (iso_code varchar(255) NOT NULL, \
                population  bigint, \
                population_density float, \
                median_age float, \
                aged_65_older float, \
                aged_70_older float, \
                gdp_per_capita float, \
                extreme_poverty float, \
                cardiovasc_death_rate float, \
                diabetes_prevalence float, \
                female_smokers float, \
                male_smokers float, \
                handwashing_facilities float, \
                hospital_beds_per_thousand float, \
                life_expectancy float, \
                human_development_index float)")

In [8]:
loc_df.population = loc_df.population.fillna(0)
loc_df.population_density = loc_df.population_density.fillna(0)
loc_df.median_age = loc_df.median_age.fillna(0)
loc_df.aged_65_older = loc_df.aged_65_older.fillna(0)
loc_df.aged_70_older = loc_df.aged_70_older.fillna(0)
loc_df.gdp_per_capita = loc_df.gdp_per_capita.fillna(0)
loc_df.extreme_poverty = loc_df.extreme_poverty.fillna(0)
loc_df.cardiovasc_death_rate = loc_df.cardiovasc_death_rate.fillna(0)
loc_df.diabetes_prevalence = loc_df.diabetes_prevalence.fillna(0)
loc_df.female_smokers = loc_df.female_smokers.fillna(0)
loc_df.male_smokers = loc_df.male_smokers.fillna(0)
loc_df.handwashing_facilities = loc_df.handwashing_facilities.fillna(0)
loc_df.hospital_beds_per_thousand = loc_df.hospital_beds_per_thousand.fillna(0)
loc_df.life_expectancy = loc_df.life_expectancy.fillna(0)
loc_df.human_development_index = loc_df.human_development_index.fillna(0)

C:\Users\tomas\anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
for index, row in loc_df.iterrows():
    cursor.execute("INSERT INTO dbo.country_data([iso_code], \
                        [population], \
                        [population_density], \
                        [median_age], \
                        [aged_65_older], \
                        [aged_70_older], \
                        [gdp_per_capita], \
                        [extreme_poverty], \
                        [cardiovasc_death_rate], \
                        [diabetes_prevalence], \
                        [female_smokers], \
                        [male_smokers], \
                        [handwashing_facilities], \
                        [hospital_beds_per_thousand], \
                        [life_expectancy], \
                        [human_development_index]) \
                    VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                    row['iso_code'],
                    row['population'],
                    row['population_density'],
                    row['median_age'],
                    row['aged_65_older'],
                    row['aged_70_older'],
                    row['gdp_per_capita'],
                    row['extreme_poverty'],
                    row['cardiovasc_death_rate'],
                    row['diabetes_prevalence'],
                    row['female_smokers'],
                    row['male_smokers'],
                    row['handwashing_facilities'],
                    row['hospital_beds_per_thousand'],
                    row['life_expectancy'],
                    row['human_development_index'])

conn.commit()

Mortality

In [10]:
cursor.execute("DROP TABLE IF EXISTS mortality; \
                CREATE TABLE mortality \
                (iso_code varchar(255), \
                date date, \
                excess_mortality float, \
                excess_mortality_cumulative float, \
                excess_mortality_cumulative_absolute float, \
                excess_mortality_cumulative_per_million float)")

In [11]:
mort_df = df[df['excess_mortality'].notna()]

In [12]:
for index, row in mort_df.iterrows():
    cursor.execute("INSERT INTO dbo.mortality(\
                        [iso_code], \
                        [date], \
                        [excess_mortality], \
                        [excess_mortality_cumulative], \
                        [excess_mortality_cumulative_absolute], \
                        [excess_mortality_cumulative_per_million]) \
                    VALUES(?, ?, ?, ?, ?, ?)",
                    row['iso_code'],
                    row['date'],
                    row['excess_mortality'],
                    row['excess_mortality_cumulative'],
                    row['excess_mortality_cumulative_absolute'],
                    row['excess_mortality_cumulative_per_million'])
conn.commit()

Covid Timeline

In [13]:
cursor.execute("DROP TABLE IF EXISTS covid_timeline; \
                CREATE TABLE covid_timeline \
                (iso_code varchar(255), \
                date date, \
                total_cases float, \
                new_cases float, \
                total_deaths float, \
                new_deaths float, \
                icu_patients float, \
                hosp_patients float, \
                total_tests float, \
                new_tests float, \
                tests_units varchar(255), \
                total_vaccinations float, \
                people_vaccinated float, \
                people_fully_vaccinated float, \
                total_boosters float, \
                new_vaccinations float, \
                stringency_index float)")

In [14]:
rslt_df.total_cases = rslt_df.total_cases.fillna(0)
rslt_df.new_cases = rslt_df.new_cases.fillna(0)
rslt_df.total_deaths = rslt_df.total_deaths.fillna(0)
rslt_df.new_deaths = rslt_df.new_deaths.fillna(0)
rslt_df.icu_patients = rslt_df.icu_patients.fillna(0)
rslt_df.hosp_patients = rslt_df.hosp_patients.fillna(0)
rslt_df.total_tests = rslt_df.total_tests.fillna(0)
rslt_df.new_tests = rslt_df.new_tests.fillna(0)
rslt_df.tests_units = rslt_df.tests_units.fillna('NULL')
rslt_df.total_vaccinations = rslt_df.total_vaccinations.fillna(0)
rslt_df.people_vaccinated = rslt_df.people_vaccinated.fillna(0)
rslt_df.people_fully_vaccinated = rslt_df.people_fully_vaccinated.fillna(0)
rslt_df.total_boosters = rslt_df.total_boosters.fillna(0)
rslt_df.new_vaccinations = rslt_df.new_vaccinations.fillna(0)
rslt_df.stringency_index = rslt_df.stringency_index.fillna(0)

In [15]:
for index, row in rslt_df.iterrows():
    cursor.execute("INSERT INTO dbo.covid_timeline(\
                        [iso_code], \
                        [date], \
                        [total_cases], \
                        [new_cases], \
                        [total_deaths], \
                        [new_deaths], \
                        [icu_patients], \
                        [hosp_patients], \
                        [total_tests], \
                        [new_tests], \
                        [tests_units], \
                        [total_vaccinations], \
                        [people_vaccinated], \
                        [people_fully_vaccinated], \
                        [total_boosters], \
                        [new_vaccinations], \
                        [stringency_index]) \
                    VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                    row['iso_code'],
                    row['date'],
                    row['total_cases'],
                    row['new_cases'],
                    row['total_deaths'],
                    row['new_deaths'],
                    row['icu_patients'],
                    row['hosp_patients'],
                    row['total_tests'],
                    row['new_tests'],
                    row['tests_units'],
                    row['total_vaccinations'],
                    row['people_vaccinated'],
                    row['people_fully_vaccinated'],
                    row['total_boosters'],
                    row['new_vaccinations'],
                    row['stringency_index'])
conn.commit()

In [16]:
# a = ['iso_code','date','total_cases','new_cases','total_deaths','new_deaths','icu_patients','hosp_patients','total_tests','new_tests','tests_units','total_vaccinations','people_vaccinated','people_fully_vaccinated','total_boosters','new_vaccinations','stringency_index',]
# for i in a:
#     print('rslt_df.' + i + ' = rslt_df.' + i + '.fillna(0)')


In [17]:
cursor.close()
conn.close()

In [18]:
# pd.set_option('display.max_columns', 70)
# df.head()